<a href="https://colab.research.google.com/github/MarinaSupiot/projet_7/blob/main/MLFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gc
from hyperopt import hp
from hyperopt import tpe
from hyperopt import fmin
from hyperopt import space_eval, STATUS_OK
import plotly.express as px
from plotly.offline import iplot

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import lightgbm as lgb
#!pip install lime
#from lime import lime_tabular

import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import pickle
import random
import re
import seaborn as sns
!pip install shap
import shap


import sklearn
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, fbeta_score

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

!pip install scikit-plot
!pip install kds

import scikitplot as skplt
import kds

import pickle
import os



import time

# Versions
print('Version des librairies utilisées :')


print('Matplotlib            : ' + mpl.__version__)
print('Missingno             : ' + msno.__version__)
print('NumPy                 : ' + np.version.full_version)
print('Pandas                : ' + pd.__version__)
print('Seaborn               : ' + sns.__version__)
print('Shap                  : ' + shap.__version__)
print('Sklearn               : ' + sklearn.__version__ )

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.7/535.7 kB 4.1 MB/s eta 0:00:00
Version des librairies utilisées :
Matplotlib            : 3.7.1
Missingno             : 0.5.2
NumPy                 : 1.23.5
Pandas                : 1.5.3
Seaborn               : 0.13.1
Shap                  : 0.44.1
Sklearn               : 1.2.2


In [ ]:
X = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_sample_set.p", "rb" ) )
y = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_sample_labels.p", "rb" ) )

In [ ]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00


DummyRegressor

In [ ]:
import mlflow
import mlflow.sklearn  # pour log le modèle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
import gc
import time  # Pour mesurer le temps

# 1. Prétraiter les données
X = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_sample_set.p", "rb" ) )
y = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_sample_labels.p", "rb" ) )

# 2. Préparer la Cross-Validation
cv = StratifiedKFold(n_splits=5)

# Hyperparamètres du DummyClassifier
strategies = ['stratified']

results = []

mlflow.set_experiment("DummyClassifier")

for strategy in strategies:
    with mlflow.start_run():
        dummy = DummyClassifier(strategy=strategy, constant=1 if strategy == 'constant' else None)

        # Mesurer le temps de traitement de fit
        start_time_fit = time.time()
        dummy.fit(X, y)
        elapsed_time_fit = time.time() - start_time_fit

        # Mesurer le temps de traitement de prédiction
        start_time_predict = time.time()
        y_prob = cross_val_predict(dummy, X, y, cv=cv, method="predict_proba")[:, 1]
        elapsed_time_predict = time.time() - start_time_predict

        y_pred = (y_prob > 0.5)

        # Calcul des scores
        auc = roc_auc_score(y, y_prob)
        acc = accuracy_score(y, y_pred)
        tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
        cost = -fp - 10 * fn + tn

        # Log les hyperparamètres, métriques et temps à MLflow
        mlflow.log_param("Strategy", strategy)
        mlflow.log_metric("AUC", auc)
        mlflow.log_metric("Accuracy", acc)
        mlflow.log_metric("Business Score", cost)
        mlflow.log_metric("Fit Time", elapsed_time_fit)
        mlflow.log_metric("Prediction Time", elapsed_time_predict)

        # Log le modèle à MLflow
        mlflow.sklearn.log_model(dummy, "dummy_model")

        # Stockage et log des courbes ROC
        fpr, tpr, thresholds = roc_curve(y, y_prob)

        plt.figure(figsize=(10, 7))
        plt.plot(fpr, tpr, label=f'AUC: {auc:.2f}')
        plt.title('ROC Curve')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend(loc='lower right')
        plt.savefig("roc_curve.png")
        plt.close()
        mlflow.log_artifact("roc_curve.png")

        # Ajouter les résultats à la liste
        results.append({
            "Strategy": strategy,
            "AUC": auc,
            "Accuracy": acc,
            "Business Score": cost
        })

# Afficher les résultats
results_df = pd.DataFrame(results)

gc.collect()

2024/02/05 18:14:05 INFO mlflow.tracking.fluent: Experiment with name 'DummyClassifier' does not exist. Creating a new experiment.


186

Adaboost

In [ ]:
model_smote = pickle.load(open("/content/drive/MyDrive/Projet 7/Adaboost/model_smote.pkl", "rb"))
model_su06 = pickle.load(open("/content/drive/MyDrive/Projet 7/Adaboost/model_su06.pkl", "rb"))
model_su04 = pickle.load(open("/content/drive/MyDrive/Projet 7/Adaboost/model_su04.pkl", "rb"))
model_su02 = pickle.load(open("/content/drive/MyDrive/Projet 7/Adaboost/model_su02.pkl", "rb"))
model_us = pickle.load(open("/content/drive/MyDrive/Projet 7/Adaboost/model_us.pkl", "rb"))


In [ ]:
import mlflow
import mlflow.sklearn  # Для логирования модели
from hyperopt import fmin, tpe, hp
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, precision_recall_curve, fbeta_score
from imblearn.over_sampling import SMOTE
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import gc
import time

# Убедитесь, что функция calculate_metrics и cout_metier определены

def cout_metier(y_true, y_pred, fn_value=-10, fp_value=-1, vp_value=0, vn_value=1):
    mat_conf = confusion_matrix(y_true, y_pred)
    vn = mat_conf[0, 0]
    fn = mat_conf[1, 0]
    fp = mat_conf[0, 1]
    vp = mat_conf[1, 1]
    J = vp * vp_value + vn * vn_value + fp * fp_value + fn * fn_value

    return J

def calculate_metrics(X, y, model):
    y_pred_proba = model.predict_proba(X)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y, y_pred_proba)
    roc_auc = [roc_auc_score(y, y_pred_proba > threshold) for threshold in thresholds]
    cout_metier_values = [cout_metier(y, y_pred_proba > threshold) for threshold in thresholds]
    fbeta_scores = [fbeta_score(y, y_pred_proba > threshold, beta=2) for threshold in thresholds]

    return thresholds, precision, recall, roc_auc, cout_metier_values, fbeta_scores

# Модифицированная функция 'optimisation_model_adaboost' с SMOTE
def optimisation_model_adaboost(n_ite_baye=10):
    mlflow.set_experiment("AdaBoostOptimization_SMOTE")

    # Подготовка данных
    X = pickle.load(open("/content/drive/MyDrive/Projet 7/val_sample_set.p", "rb"))
    y = pickle.load(open("/content/drive/MyDrive/Projet 7/val_sample_labels.p", "rb"))

    # Определение пространства поиска гиперпараметров
    space = {
        'n_estimators': hp.quniform('n_estimators', 200, 2000, 200),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
        'algorithm': hp.choice('algorithm', ['SAMME', 'SAMME.R']),
        'seuil': hp.quniform('seuil', 0.0, 1.0, 0.025)
    }

    # Функция для оптимизации
    def optimize_rf(params):
        with mlflow.start_run():
            # Настройка модели
            model = AdaBoostClassifier(
                n_estimators=int(params['n_estimators']),
                learning_rate=params['learning_rate'],
                algorithm=params['algorithm'],
                random_state=42
            )

            # Разделение данных для кросс-валидации
            cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

            auc_scores = []
            business_scores = []

            for train_idx, test_idx in cv.split(X, y):
                X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
                y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

                # Применение SMOTE к тренировочным данным
                smote = SMOTE(random_state=42, k_neighbors=10)
                X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
                # Обучение модели
                model.fit(X_train_smote, y_train_smote)

                # Предсказания
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                y_pred = (y_pred_proba > params['seuil']).astype(int)

                # Расчет метрик
                auc_score = roc_auc_score(y_test, y_pred_proba)
                business_score = cout_metier(y_test, y_pred)
                auc_scores.append(auc_score)
                business_scores.append(business_score)

            # Средние метрики
            avg_auc = np.mean(auc_scores)
            avg_business_score = np.mean(business_scores)

            # Логирование параметров и метрик
            mlflow.log_params(params)
            mlflow.log_metric("AUC", avg_auc)
            mlflow.log_metric("Business Score", avg_business_score)

            # Логирование модели
            mlflow.sklearn.log_model(model, "AdaBoostModel_SMOTE")

            return 1 - avg_auc  # Минимизируем 1 - средний AUC

    # Запуск оптимизации
    best_params = fmin(fn=optimize_rf, space=space, algo=tpe.suggest, max_evals=n_ite_baye)

    return best_params

# Запуск оптимизации с SMOTE
best_params = optimisation_model_adaboost(n_ite_baye=10)
print("Лучшие параметры:", best_params)

LighGBM

In [ ]:
import mlflow
import mlflow.sklearn  # Для логирования модели, хотя для LightGBM мы используем другой метод
from hyperopt import fmin, tpe, hp
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, precision_recall_curve, fbeta_score
from imblearn.over_sampling import SMOTE
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import gc
import time

# Убедитесь, что функция calculate_metrics и cout_metier определены

def cout_metier(y_true, y_pred, fn_value=-10, fp_value=-1, vp_value=0, vn_value=1):
    mat_conf = confusion_matrix(y_true, y_pred)
    vn = mat_conf[0, 0]
    fn = mat_conf[1, 0]
    fp = mat_conf[0, 1]
    vp = mat_conf[1, 1]
    J = vp * vp_value + vn * vn_value + fp * fp_value + fn * fn_value

    return J

def calculate_metrics(X, y, model):
    y_pred_proba = model.predict_proba(X)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y, y_pred_proba)
    roc_auc = [roc_auc_score(y, y_pred_proba > threshold) for threshold in thresholds]
    cout_metier_values = [cout_metier(y, y_pred_proba > threshold) for threshold in thresholds]
    fbeta_scores = [fbeta_score(y, y_pred_proba > threshold, beta=2) for threshold in thresholds]

    return thresholds, precision, recall, roc_auc, cout_metier_values, fbeta_scores

# Модифицированная функция 'optimisation_model_adaboost' с SMOTE
def optimisation_model_lightgbm(n_ite_baye=10):
    mlflow.set_experiment("LightGBMOptimization_SMOTE")

    # Подготовка данных
    X = pickle.load(open("/content/drive/MyDrive/Projet 7/val_sample_set.p", "rb"))
    y = pickle.load(open("/content/drive/MyDrive/Projet 7/val_sample_labels.p", "rb"))

    # Определение пространства поиска гиперпараметров
    space = {
        'n_estimators': hp.quniform('n_estimators', 200, 2000, 200),
        'max_depth': hp.quniform('max_depth', 2, 30, 2),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
        'subsample': hp.quniform('subsample', 0.1, 1.0, 0.1),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 1.0, 0.1),
        'num_leaves': hp.quniform('num_leaves', 4, 100, 4),
        'reg_alpha': hp.quniform('reg_alpha', 0.1, 1.0, 0.1),
        'reg_lambda': hp.quniform('reg_lambda', 0.1, 1.0, 0.1),
        'seuil': hp.quniform('seuil', 0.0, 1.0, 0.025)
    }

    # Функция для оптимизации
    def optimize_lgbm(params):
        with mlflow.start_run():
            # Настройка модели
            model = LGBMClassifier(
                n_estimators=int(params['n_estimators']),
                max_depth=int(params['max_depth']),
                learning_rate=params['learning_rate'],
                subsample=params['subsample'],
                colsample_bytree=params['colsample_bytree'],
                num_leaves=int(params['num_leaves']),
                reg_alpha=params['reg_alpha'],
                reg_lambda=params['reg_lambda'],
                random_state=42
            )

            cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

            auc_scores = []
            business_scores = []

            for train_idx, test_idx in cv.split(X, y):
                X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
                y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

                # Применение SMOTE к тренировочным данным
                smote = SMOTE(random_state=42)
                X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

                # Обучение модели
                model.fit(X_train_smote, y_train_smote)

                # Предсказания
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                y_pred = (y_pred_proba > params['seuil']).astype(int)

                # Расчет метрик
                auc_score = roc_auc_score(y_test, y_pred_proba)
                business_score = cout_metier(y_test, y_pred)
                auc_scores.append(auc_score)
                business_scores.append(business_score)

            # Средние метрики
            avg_auc = np.mean(auc_scores)
            avg_business_score = np.mean(business_scores)

            # Логирование параметров и метрик
            mlflow.log_params(params)
            mlflow.log_metric("AUC", avg_auc)
            mlflow.log_metric("Business Score", avg_business_score)

            # Логирование модели
            mlflow.sklearn.log_model(model, "LightGBMModel_SMOTE")  # Используйте mlflow.lightgbm.log_model для LightGBM

            return {'loss': 1 - avg_auc, 'status': STATUS_OK}  # Минимизируем 1 - средний AUC

    # Запуск оптимизации
    best_params = fmin(fn=optimize_lgbm, space=space, algo=tpe.suggest, max_evals=n_ite_baye)

    return best_params

# Запуск оптимизации с SMOTE для LightGBM
best_params = optimisation_model_lightgbm(n_ite_baye=10)
print("Лучшие параметры:", best_params)

2024/02/05 18:28:05 INFO mlflow.tracking.fluent: Experiment with name 'LightGBMOptimization_SMOTE' does not exist. Creating a new experiment.


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

In [ ]:
import subprocess

# Запустить сервер MLflow в фоновом режиме
command = ["mlflow", "server", "--host", "127.0.0.1", "--port", "5000"]
process = subprocess.Popen(command)

# Проверьте, что процесс запущен (process.pid даст вам PID процесса, если он запущен)
print(f"MLflow server is running with PID: {process.pid}")


MLflow server is running with PID: 59327


In [ ]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Инициализация эксперимента
mlflow.set_experiment("DummyClassifier")

<Experiment: artifact_location='file:///content/mlruns/468158438677165734', creation_time=1706880163274, experiment_id='468158438677165734', last_update_time=1706880163274, lifecycle_stage='active', name='DummyClassifier', tags={}>

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2YGm2G54kBEMvnAj43IFcf4QK1O_7kd3J9UnbA4TGcvw9W5Np"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://d8c1-35-196-132-161.ngrok-free.app


In [ ]:
!mlflow ui

[2024-02-05 20:36:31 +0000] [36535] [INFO] Starting gunicorn 21.2.0
[2024-02-05 20:36:31 +0000] [36535] [INFO] Listening at: http://127.0.0.1:5000 (36535)
[2024-02-05 20:36:31 +0000] [36535] [INFO] Using worker: sync
[2024-02-05 20:36:31 +0000] [36536] [INFO] Booting worker with pid: 36536
[2024-02-05 20:36:31 +0000] [36537] [INFO] Booting worker with pid: 36537
[2024-02-05 20:36:31 +0000] [36538] [INFO] Booting worker with pid: 36538
[2024-02-05 20:36:31 +0000] [36539] [INFO] Booting worker with pid: 36539
